In [2]:
import pandas as pd

In [4]:
with open('datasets/data_raw/dailydialog/dialogues_text.txt', encoding= 'utf-8') as f:
    dialogue = f.readlines()

with open('datasets/data_raw/dailydialog/dialogues_emotion.txt', encoding= 'utf-8') as f:
    emotion = f.readlines()

d=[]
sentences = []
for line in dialogue:
    line = line.replace('\n', '')
    d.append(line.split('__eou__'))

for lis in d:
    for sentence in lis:
        sentences.append(sentence)

sentences = list(filter(None,sentences))

emotion = ' '.join(emotion).split()

df = pd.DataFrame(zip(emotion, sentences), columns=['emotion','text'])
df.to_csv('datasets/dailydialog.csv')
df.head()


,emotion,text
0,2,The kitchen stinks .
1,0,I'll throw out the garbage .
2,4,"So Dick , how about getting some coffee for to..."
3,2,Coffee ? I don ’ t honestly like that kind of...
4,0,"Come on , you can at least try a little , bes..."


In [25]:
with open('datasets/data_raw/emotion_stimulus/Emotion Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus = f.readlines()

with open('datasets/data_raw/emotion_stimulus/No Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus_no_cause = f.readlines()

emotion_stimulus_all = emotion_stimulus+emotion_stimulus_no_cause
print(len(emotion_stimulus_all))

2414


In [26]:
emotion_stimulus_all[:5]

["<happy>I suppose I am happy <cause>being so ` tiny'<\\cause> ; it means I am able to surprise people with what is generally seen as my confident and outgoing personality . <\\happy>\n",
 '<happy>Lennox has always truly wanted to fight for the world title and was happy <cause>taking the tough route<\\cause> . <\\happy>\n',
 '<happy>He was a professional musician now , still sensitive and happy <cause>doing something he loved<\\cause> . <\\happy>\n',
 '<happy>Holmes is happy <cause>having the freedom of the house when we are out<\\cause> . <\\happy>\n',
 '<happy>I had problems with tutors trying to encourage me to diversity my work and experiment with other styles , but I was quite happy <cause>with the direction my work was heading<\\cause> so I stubbornly stuck to it . <\\happy>\n']

In [60]:
df_emotion_stimulus = pd.DataFrame(emotion_stimulus_all,columns=['text'])
df_emotion_stimulus.head()


,text
0,<happy>I suppose I am happy <cause>being so ` ...
1,<happy>Lennox has always truly wanted to fight...
2,"<happy>He was a professional musician now , st..."
3,<happy>Holmes is happy <cause>having the freed...
4,<happy>I had problems with tutors trying to en...


In [61]:
df_emotion_stimulus['emotion'] = df_emotion_stimulus['text'].apply(lambda x: str(x.split('>')[0]))

df_emotion_stimulus =  df_emotion_stimulus.applymap(lambda x: str(x.replace('<cause>','')))

In [62]:
df_emotion_stimulus.head()

,text,emotion
0,<happy>I suppose I am happy being so ` tiny'<\...,<happy
1,<happy>Lennox has always truly wanted to fight...,<happy
2,"<happy>He was a professional musician now , st...",<happy
3,<happy>Holmes is happy having the freedom of t...,<happy
4,<happy>I had problems with tutors trying to en...,<happy
